In [1]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit
import pandas as pd
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
RANDOM_STATE = 42
N_PREDICTIONS = 100

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [3]:
df_vacancies = pd.read_csv('/Users/anastasiakucina/hahaha/vacancies_ru.csv')
df_vacancies.head()

,Unnamed: 0,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder
0,0,v_862116,Смотритель музейный,c_162972,"['Пользователь ПК', 'Работа в команде', 'Умени...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1
1,1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1
2,2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,NaN,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5,3
3,3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,NaN,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5,1
4,4,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2


In [4]:
df_vacancies_final = df_vacancies[['vacancy_id', 'area_regionId_encoder', 'area_id_encoder', 'company_id_encoder', 'employment_encoder', 'workSchedule_encoder', 'workExperience_encoder', 'compensation_from']]
df_vacancies_final.head()

,vacancy_id,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,compensation_from
0,v_862116,32,4181,69972,5,5,1,16500.0
1,v_288642,17,6731,120750,5,5,1,50000.0
2,v_1840054,81,5806,109013,5,5,3,50000.0
3,v_2346232,52,4218,235992,5,5,1,38500.0
4,v_312507,39,6487,118557,5,5,2,60000.0


In [5]:
train = pd.read_csv('/Users/anastasiakucina/hahaha/train_test.csv')
train.head()

,Unnamed: 0,user_id,session_id,vacancy_id,action_type,action_dt,clear_vacancy_id
0,0,u_482520,s_25018731,"['v_2597196', 'v_1223061', 'v_1223061']","[2, 2, 1]","[1700395393089, 1700395410396, 1700395503473]","['v_2597196', 'v_1223061', 'v_1223061']"
1,2,u_212584,s_16918781,"['v_1572055', 'v_1572055', 'v_1572055', 'v_953...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[1700124107031, 1700124321403, 1700124426230, ...","['v_1572055', 'v_1572055', 'v_1572055', 'v_953..."
2,3,u_425177,s_17505104,"['v_1375331', 'v_1922852']","[2, 2]","[1700224938513, 1700225032744]","['v_1375331', 'v_1922852']"
3,4,u_700997,s_15528830,"['v_2152997', 'v_2152997', 'v_1217630']","[2, 1, 2]","[1700155373530, 1700155399885, 1700155413472]","['v_2152997', 'v_2152997', 'v_1217630']"
4,13,u_186164,s_34273595,"['v_2674644', 'v_2558182']","[2, 2]","[1700483122760, 1700483363810]","['v_2674644', 'v_2558182']"


In [6]:
df_vacancies_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2209572 entries, 0 to 2209571
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   vacancy_id              object 
 1   area_regionId_encoder   int64  
 2   area_id_encoder         int64  
 3   company_id_encoder      int64  
 4   employment_encoder      int64  
 5   workSchedule_encoder    int64  
 6   workExperience_encoder  int64  
 7   compensation_from       float64
dtypes: float64(1), int64(6), object(1)
memory usage: 134.9+ MB


In [7]:
def str_to_list(row):
    return eval(row)

In [8]:
train['vacancy_id_list'] = train['vacancy_id'].apply(str_to_list)
train['action_type_list'] = train['action_type'].apply(str_to_list)
train.head()

,Unnamed: 0,user_id,session_id,vacancy_id,action_type,action_dt,clear_vacancy_id,vacancy_id_list,action_type_list
0,0,u_482520,s_25018731,"['v_2597196', 'v_1223061', 'v_1223061']","[2, 2, 1]","[1700395393089, 1700395410396, 1700395503473]","['v_2597196', 'v_1223061', 'v_1223061']","[v_2597196, v_1223061, v_1223061]","[2, 2, 1]"
1,2,u_212584,s_16918781,"['v_1572055', 'v_1572055', 'v_1572055', 'v_953...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]","[1700124107031, 1700124321403, 1700124426230, ...","['v_1572055', 'v_1572055', 'v_1572055', 'v_953...","[v_1572055, v_1572055, v_1572055, v_953153, v_...","[2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1]"
2,3,u_425177,s_17505104,"['v_1375331', 'v_1922852']","[2, 2]","[1700224938513, 1700225032744]","['v_1375331', 'v_1922852']","[v_1375331, v_1922852]","[2, 2]"
3,4,u_700997,s_15528830,"['v_2152997', 'v_2152997', 'v_1217630']","[2, 1, 2]","[1700155373530, 1700155399885, 1700155413472]","['v_2152997', 'v_2152997', 'v_1217630']","[v_2152997, v_2152997, v_1217630]","[2, 1, 2]"
4,13,u_186164,s_34273595,"['v_2674644', 'v_2558182']","[2, 2]","[1700483122760, 1700483363810]","['v_2674644', 'v_2558182']","[v_2674644, v_2558182]","[2, 2]"


In [9]:
df_vacancy_id = train[['user_id', 'vacancy_id_list']].explode('vacancy_id_list')
df_pairs = train[['user_id', 'action_type_list']].explode('action_type_list')
df_pairs

,user_id,action_type_list
0,u_482520,2
0,u_482520,2
0,u_482520,1
1,u_212584,2
1,u_212584,3
...,...,...
38578,u_847738,2
38578,u_847738,1
38578,u_847738,2
38578,u_847738,2


In [10]:
df_pairs['vacancy_id'] = df_vacancy_id['vacancy_id_list']
df_pairs

,user_id,action_type_list,vacancy_id
0,u_482520,2,v_2597196
0,u_482520,2,v_1223061
0,u_482520,1,v_1223061
1,u_212584,2,v_1572055
1,u_212584,3,v_1572055
...,...,...,...
38578,u_847738,2,v_1302840
38578,u_847738,1,v_1302840
38578,u_847738,2,v_1247305
38578,u_847738,2,v_1764585


In [11]:
def drop_vu(row):
    return row.split('_')[1]

In [12]:
df_pairs['vacancy_id'] =  df_pairs['vacancy_id'].apply(drop_vu)
df_pairs['user_id'] = df_pairs['user_id'].apply(drop_vu)

In [13]:
df_pairs

,user_id,action_type_list,vacancy_id
0,482520,2,2597196
0,482520,2,1223061
0,482520,1,1223061
1,212584,2,1572055
1,212584,3,1572055
...,...,...,...
38578,847738,2,1302840
38578,847738,1,1302840
38578,847738,2,1247305
38578,847738,2,1764585


### Train&Test KNN



In [14]:

from catboost import CatBoostClassifier, Pool


In [15]:
# Train on half of users, because of kernel;(
df_pairs_half = df_pairs[:99000]
vacancies_uniq = list(df_pairs_half['vacancy_id'].unique())

In [16]:
vacancies_uniq

['2597196',
 '1223061',
 '1572055',
 '953153',
 '1495772',
 '1033603',
 '1592343',
 '1329494',
 '1375331',
 '1922852',
 '2152997',
 '1217630',
 '2674644',
 '2558182',
 '2171209',
 '607672',
 '1401407',
 '1977347',
 '2097290',
 '2373949',
 '1529417',
 '2666974',
 '1893426',
 '1240196',
 '102362',
 '1794489',
 '125123',
 '136431',
 '2299253',
 '645319',
 '533956',
 '2220991',
 '2676265',
 '1579567',
 '381847',
 '2603095',
 '2210018',
 '125978',
 '1378565',
 '2708324',
 '2445762',
 '1881691',
 '1967337',
 '1477939',
 '1106794',
 '694434',
 '1402265',
 '1823852',
 '1298290',
 '682939',
 '1720648',
 '772446',
 '1656647',
 '1752086',
 '1572049',
 '81466',
 '742925',
 '2047167',
 '1987747',
 '285124',
 '2122905',
 '1940171',
 '22704',
 '437305',
 '676401',
 '1586276',
 '960735',
 '1921993',
 '1803280',
 '1026899',
 '1723789',
 '634398',
 '1305351',
 '505259',
 '1428978',
 '2530191',
 '1040904',
 '482905',
 '1339656',
 '2559035',
 '389728',
 '2008839',
 '1595640',
 '596382',
 '2350075',
 '4973

In [17]:
df_vacancies_final['vacancy_id'] = df_vacancies_final['vacancy_id'].apply(drop_vu)
df_vacancies_final

/var/folders/gz/46_zg8xd6qqgn1sxylfq7g8m0000gn/T/ipykernel_39616/3594289813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacancies_final['vacancy_id'] = df_vacancies_final['vacancy_id'].apply(drop_vu)


,vacancy_id,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,compensation_from
0,862116,32,4181,69972,5,5,1,16500.0
1,288642,17,6731,120750,5,5,1,50000.0
2,1840054,81,5806,109013,5,5,3,50000.0
3,2346232,52,4218,235992,5,5,1,38500.0
4,312507,39,6487,118557,5,5,2,60000.0
...,...,...,...,...,...,...,...,...
2209567,2484959,74,4876,114733,5,2,1,40000.0
2209568,205163,71,5032,65219,5,5,2,80000.0
2209569,639897,41,842,121520,5,5,2,200000.0
2209570,1636531,62,2673,162497,5,5,1,25000.0


In [18]:

df_vacancies_final=df_vacancies_final.query('vacancy_id in @vacancies_uniq')

In [ ]:

# Преобразование категориальных признаков в числовые
#df_pairs_half['action_type_list'] = df_pairs_half['action_type_list'].astype('category').cat.codes

# Создание матрицы пользователь-вакансия с учетом доли каждой вакансии
user_vacancy_matrix = pd.pivot_table(df_pairs_half, values='action_type_list', index='user_id', columns='vacancy_id', fill_value=0)

# Преобразование долей вакансий в веса
user_vacancy_matrix = user_vacancy_matrix.div(user_vacancy_matrix.sum(axis=1), axis=0)

# Добавление фичей из vacancies.csv -- пока тяжеляк 
df_merged = pd.merge(df_pairs_half, df_vacancies_final, on='vacancy_id')


# Преобразование данных в формат CatBoost
cat_features = ['vacancy_id', 'area_id_encoder', 'workExperience_encoder']
train_pool = Pool(data=df_merged.drop(columns=['action_type_list']), label=df_merged['action_type_list'], cat_features=cat_features)

# Обучение модели CatBoost
model = CatBoostClassifier(iterations=100, learning_rate=0.1, loss_function='MultiClass')
model.fit(train_pool)



: 

In [ ]:
pd.merge(df_pairs_half, df_vacancies_final, on='vacancy_id')

In [ ]:

# Вывод рекомендаций
for user_id, vacancies in user_vacancy_matrix.iterrows():
    user_data = df_pairs_half
    user_pool = Pool(data=user_data, cat_features=['user_id', 'vacancy_id'])
    user_predictions = model.predict(user_pool)
    recommended_vacancies = user_data.iloc[user_predictions.argmax(axis=1)]['vacancy_id']
    print(f"Для пользователя {user_id} рекомендуются вакансии с vacancy_id:", end=" ")
    print(recommended_vacancies[:5].tolist())  # Вывод не более 10 рекомендаций
    